<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Understanding-TensorFlow-2.x" data-toc-modified-id="Understanding-TensorFlow-2.x-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Understanding TensorFlow 2.x</a></span><ul class="toc-item"><li><span><a href="#eager-execution," data-toc-modified-id="eager-execution,-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>eager execution,</a></span></li></ul></li><li><span><a href="#Keras-APIs-–-three-programming-models" data-toc-modified-id="Keras-APIs-–-three-programming-models-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Keras APIs – three programming models</a></span><ul class="toc-item"><li><span><a href="#Sequential-API" data-toc-modified-id="Sequential-API-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Sequential API</a></span></li><li><span><a href="#Functional-API" data-toc-modified-id="Functional-API-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Functional API</a></span></li><li><span><a href="#Model-subclassing" data-toc-modified-id="Model-subclassing-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Model subclassing</a></span></li></ul></li><li><span><a href="#Converting-from-1.x-to-2.x" data-toc-modified-id="Converting-from-1.x-to-2.x-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Converting from 1.x to 2.x</a></span></li><li><span><a href="#Using-TensorFlow-2.x-effectively" data-toc-modified-id="Using-TensorFlow-2.x-effectively-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Using TensorFlow 2.x effectively</a></span></li><li><span><a href="#Callbacks" data-toc-modified-id="Callbacks-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Callbacks</a></span></li></ul></div>

# Understanding TensorFlow 2.x
## eager execution,
meaning that the model definitions are dynamic, and the execution is immediate.
Graphs and sessions should be considered as implementation details. The good news is that TensorFlow 2.x natively supports "eager execution."
There is no longer the need to first statically define a computational graph and
then execute it (unless you really wanted to!). All the models can be dynamically
defined and immediately executed. 

AutoGraph comes into play: AutoGraph takes eager-style Python code
and automatically converts it to graph-generating code.

# Keras APIs – three programming models
## Sequential API

Sequential API when we discussed the MNIST code.

## Functional API

In [6]:
import tensorflow as tf
from keras.utils import plot_model


def build_model():
    # variable-length sequence of integers
    text_input_a = tf.keras.Input(shape=(None, ), dtype='int32')
    # variable-length sequence of integers
    text_input_b = tf.keras.Input(shape=(None, ), dtype='int32')
    # Embedding for 1000 unique words mapped to 128-dimensional vectors
    shared_embedding = tf.keras.layers.Embedding(1000, 128)
    # We reuse the same layer to encode both inputs
    encoded_input_a = shared_embedding(text_input_a)
    encoded_input_b = shared_embedding(text_input_b)
    # two logistic predictions at the end
    prediction_a = tf.keras.layers.Dense(1,
                                         activation='sigmoid',
                                         name='prediction_a')(encoded_input_a)
    prediction_b = tf.keras.layers.Dense(1,
                                         activation='sigmoid',
                                         name='prediction_b')(encoded_input_b)
    # this model has 2 inputs, and 2 outputs
    # in the middle we have a shared model
    model = tf.keras.Model(inputs=[text_input_a, text_input_b],
                           outputs=[prediction_a, prediction_b])
#     tf.keras.utils.plot_model(model, to_file="./shared_model.png")


build_model()

<img src="./i/shared_model.png" />

## Model subclassing

Model subclassing offers the highest flexibility and it is generally used when
you need to define your own layer. In other words, it is useful when you are in
the business of building your own special lego brick instead of composing more
standard and well-known bricks.

    • __init__: Optionally used to define all the sublayers to be used by this layer.
    This is the constructor where you can declare your model.
    • build: Used to create the weights of the layer. You can add weights with
    add_weight().
    • call: Used to define the forward pass. This is where your layer is called and
    chained in functional style.
    • Optionally, a layer can be serialized by using get_config() and deserialized
    using from_config().

In [8]:
class MyLayer(layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)
    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel',
        shape=(input_shape[1], self.output_dim),
        initializer='uniform',
        trainable=True)
    def call(self, inputs):
    # Do the multiplication and return
        return tf.matmul(inputs, self.kernel)
    
    
model = tf.keras.Sequential([
            MyLayer(20),
            layers.Activation('softmax')])

# Converting from 1.x to 2.x
TensorFlow 1.x scripts will not work directly with TensorFlow 2.x but they need
converting. The first step to convert from 1.x to 2.x is to use the automatic conversion
script installed with 2.x. For a single file, you can run it with:

tf_upgrade_v2 --infile tensorfoo.py --outfile tensorfoo-upgraded.py

For multiple files in a directory, the syntax is:

tf_upgrade_v2 --intree incode --outtree code-upgraded

The script will try to upgrade automatically to 2.x and will print error messages
where it is not able to upgrade.

# Using TensorFlow 2.x effectively
2.x native code should follow a number of best practices:

    1. Default to higher-level APIs such as tf.keras (or in certain situations,
    Estimators) and avoid lower-level APIs with direct computational graph
    manipulation unless needed for custom operations. So, in general, no
    tf.Session, tf.Session.run.
    
    2. Add a tf.function decorator to make it run efficiently in graph mode with
    AutoGraph. Only use tf.function to decorate high-level computations; all
    functions invoked by high-level computations are automatically annotated
    on your behalf. In this way, you get the best of both worlds: high-level APIs
    with eager support, and the efficiency of computational graphs.
    
    3. Use Python objects to track variables and losses. So, be Pythonic and use
    tf.Variable instead of tf.get_variable. In this way, variables will be
    treated with the normal Python scope.
    
    4. Use tf.data datasets for data inputs and provide these objects directly
    to tf.keras.Model.fit. In this way, you will have a collection of highperformance classes for manipulating data and will adopt the best way to
    stream training data from disk.
    
    5. Use tf.layers modules to combine predefined "lego bricks" whenever it
    is possible, either with Sequential or Functional APIs, or with Subclassing.
    Use Estimators if you need to have production-ready models, in particular
    if these models need to scale on multiple GPUs, CPUs, or on multiple servers.
    When needed, consider converting a tf.keras model into an Estimator.
    
    6. Consider using a distribution strategy across GPUs, CPUs, and multiple
    servers. With tf.keras it is easy

# Callbacks

In [ ]:
67 (102 / 647)